In [0]:
print('hello world')

hello world


In [0]:
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName("Datasetexploration").getOrCreate()

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df = spark.read.table("dataset_2017_fordgobike_tripdata_1_csv")

In [0]:
df.show(3)

+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------+-----------------+-------------+-----------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id|user_type|member_birth_year|member_gender|     pyment|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------+-----------------+-------------+-----------+
|   57:39.7| 12:50.2|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96| Customer|             1987|         Male|credit card|
|   56:3

In [0]:
df.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [0]:
df.count()

Out[11]: 519700

In [0]:
# import packages 
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types


def getNullNanBlankCount(df: DataFrame) -> DataFrame: 
  
  # get all the column names into a list
  allColsList = df.columns
  
  # create an empty list to temporarily store column names and count values
  listBuffer = []
  
  for field in allColsList:   
    # count nulls
    nullCondition = F.isnull(F.col(field))
    nullCount = df.select(F.col(field)).filter(nullCondition).count()  
    
    # count nans
    nanCondition = F.isnan(F.col(field))
    nanCount = df.select(F.col(field)).filter(nanCondition).count()  
    
    # count blank/empty values
    blankCount = df.select(F.col(field)).filter(F.col(field) == " ").count()  
     
    # append column names and count values to the list
    listBuffer.append((field, nullCount, nanCount, blankCount))

  # convert listBuffer into a new dataframe
  resDf = spark.createDataFrame(listBuffer, ["table_column_name", "null_count", "nan_count", "blank_count"])
 
  return resDf
  
  

In [0]:
df.transform(getNullNanBlankCount).show(truncate=False)

+-----------------------+----------+---------+-----------+
|table_column_name      |null_count|nan_count|blank_count|
+-----------------------+----------+---------+-----------+
|start_time             |0         |0        |0          |
|end_time               |0         |0        |0          |
|start_station_id       |0         |0        |0          |
|start_station_name     |0         |0        |0          |
|start_station_latitude |0         |0        |0          |
|start_station_longitude|0         |0        |0          |
|end_station_id         |0         |0        |0          |
|end_station_name       |0         |0        |0          |
|end_station_latitude   |0         |0        |0          |
|end_station_longitude  |0         |0        |0          |
|bike_id                |0         |0        |0          |
|user_type              |0         |0        |0          |
|member_birth_year      |66541     |0        |0          |
|member_gender          |66462     |0        |0         

We can easily notice that there are only 2 columns with nan values and we will see how to deal with them later on

In [0]:
df.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [0]:
df1 = df

In [0]:
df1.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [0]:
df1.filter(df1['member_birth_year'].isNull()).count()

Out[7]: 66541

## 1. Looking for a way of getting the distance travelled for each trip

In [0]:

df1 = df1.withColumn('start_station_latitude',df['start_station_latitude'].cast("double")).\
         withColumn('start_station_longitude',df['start_station_longitude'].cast("double")).\
        withColumn('end_station_latitude',df['end_station_latitude'].cast("double")).\
        withColumn('end_station_longitude',df['end_station_longitude'].cast("double"))

In [0]:
df1.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [0]:
pip install haversine

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import haversine
from haversine import haversine, Unit


In [0]:
from pyspark.sql.functions import udf , round
from pyspark.sql.types import StringType,DoubleType
# defining the function you want 
def distance(lat1,long1,lat2,long2):
    loc1 = (lat1,long1)
    loc2 = (lat2,long2)
    distance1 =haversine(loc1,loc2, Unit.KILOMETERS)
    return distance1

distanceUDF = udf(distance , DoubleType())

In [0]:
df1 = df1.withColumn("distance_travelled", distanceUDF(df1['start_station_latitude'],df1['start_station_longitude'],df1['end_station_latitude'] ,df1['end_station_longitude']))#.select('distance_travelled').show()
                                #.withColumn('distance_travelled',round(df1['distance_travelled'],2)).select('distance_travelled').show()                                                            
                        

In [0]:
df1.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_travelled: double (nullable = true)



In [0]:
df1 = df1.withColumn('distance_travelled',round(df1['distance_travelled'],2))#.select('distance_travelled').show() 

In [0]:
df1.select('distance_travelled').show() 

+------------------+
|distance_travelled|
+------------------+
|              0.94|
|              3.07|
|               0.0|
|              1.05|
|              0.64|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
|              0.77|
|              0.77|
|              1.52|
|              1.52|
|              1.42|
|              1.05|
|              1.05|
|              2.86|
|              2.86|
|              2.86|
+------------------+
only showing top 20 rows



## 2. let us now see if we can get the time taken by each trip

In [0]:
from pyspark.sql.functions import *
df1 = df1.withColumn('start_time', to_timestamp(col('start_time'),"HH:mm:ss.SSS"))\
    .withColumn('end_time', to_timestamp(col('end_time'),"HH:mm:ss.SSS"))

In [0]:
df1.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_travelled: double (nullable = true)



In [0]:
df1.withColumn('trip_time', df1['start_time'] - df1['end_time']).select('trip_time').show()

+---------+
|trip_time|
+---------+
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
+---------+
only showing top 20 rows



In [0]:
df.select('start_time').show()

+----------+
|start_time|
+----------+
|   57:39.7|
|   56:34.8|
|   45:48.4|
|   31:10.6|
|   23:14.0|
|   51:00.9|
|   49:28.4|
|   46:37.2|
|   37:07.5|
|   35:38.1|
|   46:32.4|
|   48:11.8|
|   52:55.6|
|   52:55.9|
|   35:23.2|
|   53:38.9|
|   54:40.1|
|   55:09.7|
|   52:49.5|
|   46:34.0|
+----------+
only showing top 20 rows



In [0]:
df1

root
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_travelled: double (nullable = true)



In [0]:
import org.apache.spark.sql.functions

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<command-2961780054634668> in <cell line: 1>()
----> 1 import org.apache.spark.sql.functions

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    169             # Import the desired module. If you’re seeing this while debugging a failed import,
    170             # look at preceding stack frames for relevant error information.
--> 171             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    172 
    173             is_root_import = thread_local._nest_level == 1

ModuleNotFoundError: No module named 'org.apache.spark.sql'

In [0]:
df1..withColumn("DiffInSeconds",col("end_time").cast(LongType) - col("start_time").cast(LongType))
      

  File "<command-2961780054634662>", line 1
    df1..withColumn("DiffInSeconds",col("end_time").cast(LongType) - col("start_time").cast(LongType))
        ^
SyntaxError: invalid syntax


In [0]:
df1.createOrReplaceTempView('df1')

In [0]:
spark.sql('SELECT * FROM df1 ').show()


+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|distance_travelled|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+
|      null|    null|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96|  C

In [0]:
spark.sql('SELECT TIMESTAMPDIFF(SECOND, start_time, end_time) AS difference FROM df1;').select('difference').show()

+----------+
|difference|
+----------+
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
+----------+
only showing top 20 rows



In [0]:
df.show()

+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|   57:39.7| 12:50.2|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96|  Customer|             1987|         Male|credit card|
|   

In [0]:
df1.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_travelled: double (nullable = true)



In [0]:
df.show()

+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|   57:39.7| 12:50.2|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96|  Customer|             1987|         Male|credit card|
|   

In [0]:
df.show()

+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|   57:39.7| 12:50.2|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96|  Customer|             1987|         Male|credit card|
|   